In [43]:
!pip install pybaseball

In [44]:
import pandas as pd
import numpy as np
import pybaseball
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix
from pybaseball import statcast
from pybaseball import playerid_lookup, statcast_pitcher

In [45]:
start_date = '2023-04-01'
end_date = '2023-04-10'
player_info = playerid_lookup('Olson', 'Matt')
player_id = player_info['key_mlbam'].values[0]
brasier_data = statcast_pitcher(start_date, end_date, player_id)
print(f"Player ID for Sean Manaea: {player_id}")

Gathering Player Data
Player ID for Sean Manaea: 621566


In [46]:
start_date = '2023-04-01'
end_date = '2023-04-10'
data_pitch_speed = statcast(start_dt=start_date, end_dt=end_date)
selected_columns = [
    'pitch_type', 'stand', 'p_throws', 
    'balls', 'strikes', 'game_year', 'outs_when_up', 'inning', 
    'pitch_number', 'bat_score', 'effective_speed', 'pfx_x',
    'pfx_z', 'batter', 'pitcher',
    'fld_score'
]
filtered_data = data_pitch_speed[selected_columns]
effective_speed_data = filtered_data[filtered_data['pitcher'] == 640455]


This is a large query, it may take a moment to complete


  0%|          | 0/10 [00:00<?, ?it/s]/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 10%|█         | 1/10 [00:01<00:14,  1.58s/it]/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 20%|██        | 2/10 [00:01<00:06,  1.20it/s]/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is de

In [47]:
# Encoding categorical features
le_stand = LabelEncoder()
le_p_throws = LabelEncoder()

# Convert categorical columns to numerical using Label Encoding
effective_speed_data['stand'] = le_stand.fit_transform(effective_speed_data['stand'].astype(str))
effective_speed_data['p_throws'] = le_p_throws.fit_transform(effective_speed_data['p_throws'].astype(str))

# One-Hot Encoding for pitch_type
ohe = OneHotEncoder(sparse_output=False)
pitch_type_ohe = ohe.fit_transform(effective_speed_data[['pitch_type']])
pitch_type_ohe_df = pd.DataFrame(pitch_type_ohe, columns=ohe.get_feature_names_out(['pitch_type']))

# Concatenate one-hot encoded pitch types with the original DataFrame
effective_speed_data = pd.concat([effective_speed_data.reset_index(drop=True), pitch_type_ohe_df], axis=1)

# Drop the original pitch_type column
effective_speed_data = effective_speed_data.drop(['pitch_type'], axis=1)

# Feature engineering - adding interaction terms between features
interaction_features = ['balls', 'strikes', 'outs_when_up', 'inning', 'pitch_number', 'bat_score', 'fld_score']
for feature1, feature2 in itertools.combinations(interaction_features, 2):
    interaction_term = f"{feature1}_x_{feature2}"
    effective_speed_data[interaction_term] = effective_speed_data[feature1] * effective_speed_data[feature2]

print(effective_speed_data.columns)
print(effective_speed_data)

# Creating separate regressors for each pitch type
pitch_type_models = {}
pitch_types = ohe.get_feature_names_out(['pitch_type'])

for pitch_type in pitch_types:
    # Filter data for the current pitch type
    pitch_type_data = effective_speed_data[effective_speed_data[pitch_type] == 1]
    X = pitch_type_data.drop(['effective_speed', 'pitcher'], axis=1)
    y = pitch_type_data['effective_speed']

    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Creating and training the Random Forest Regressor
    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_regressor.fit(X_train, y_train)
    pitch_type_models[pitch_type] = rf_regressor

# Creating Random Forest Classifier for pitch type prediction
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
y_pitch = effective_speed_data[pitch_types].idxmax(axis=1)
X_train_pitch, X_test_pitch, y_train_pitch, y_test_pitch = train_test_split(effective_speed_data.drop(['effective_speed', 'pitcher'], axis=1), y_pitch, test_size=0.2, random_state=42)
rf_classifier.fit(X_train_pitch, y_train_pitch)

# Defining manual input values
input_data = pd.DataFrame([{
    'stand': le_stand.transform(['R'])[0] if 'R' in le_stand.classes_ else -1,
    'p_throws': le_p_throws.transform(['L'])[0] if 'L' in le_p_throws.classes_ else -1,
    'balls': 0,
    'strikes': 0,
    'game_year': 2023,
    'outs_when_up': 2,
    'inning': 9,
    'pitch_number': 0,
    'bat_score': 3,
    'fld_score': 3,
    'pfx_x': 0.2,
    'pfx_z': 0.2,
    'batter': 621566
}])

# Adding one-hot encoded pitch types with default values of 0
for col in pitch_types:
    input_data[col] = 0

# Adding interaction terms to input_data
for feature1, feature2 in itertools.combinations(interaction_features, 2):
    interaction_term = f"{feature1}_x_{feature2}"
    input_data[interaction_term] = input_data[feature1] * input_data[feature2]

# Ensuring input_data has all columns that were used during training
missing_cols = set(effective_speed_data.drop(['effective_speed', 'pitcher'], axis=1).columns) - set(input_data.columns)
for col in missing_cols:
    input_data[col] = 0
input_data = input_data[effective_speed_data.drop(['effective_speed', 'pitcher'], axis=1).columns]

# Predicting pitch type probabilities
pitch_type_probabilities = rf_classifier.predict_proba(input_data)[0]
pitch_type_probs = dict(zip(pitch_types, pitch_type_probabilities))

# Predicting effective speed for each pitch type (using separate models for each pitch type)
predicted_speeds = {}
for pitch_type in pitch_types:
    input_data_copy = input_data.copy()
    input_data_copy[pitch_type] = 1  # Set the specific pitch type to 1
    model = pitch_type_models[pitch_type]
    effective_speed_prediction = model.predict(input_data_copy)[0]
    predicted_speeds[pitch_type] = effective_speed_prediction

# Creating a DataFrame for the predictions
def create_prediction_dataframe(pitch_type_probs, predicted_speeds):
    df = pd.DataFrame({
        'Pitch Type': list(pitch_type_probs.keys()),
        'Probability': list(pitch_type_probs.values()),
        'Predicted Effective Speed': list(predicted_speeds.values())
    })
    return df

# Generating the prediction DataFrame
prediction_df = create_prediction_dataframe(pitch_type_probs, predicted_speeds)

print("Prediction DataFrame:")
print(prediction_df)



/var/folders/bc/69frl30s4xv269mkcmjkv2qr0000gn/T/ipykernel_95186/2050881682.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effective_speed_data['stand'] = le_stand.fit_transform(effective_speed_data['stand'].astype(str))
/var/folders/bc/69frl30s4xv269mkcmjkv2qr0000gn/T/ipykernel_95186/2050881682.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effective_speed_data['p_throws'] = le_p_throws.fit_transform(effective_speed_data['p_throws'].astype(str))


Index(['stand', 'p_throws', 'balls', 'strikes', 'game_year', 'outs_when_up',
       'inning', 'pitch_number', 'bat_score', 'effective_speed', 'pfx_x',
       'pfx_z', 'batter', 'pitcher', 'fld_score', 'pitch_type_CH',
       'pitch_type_FF', 'pitch_type_SI', 'pitch_type_SL', 'balls_x_strikes',
       'balls_x_outs_when_up', 'balls_x_inning', 'balls_x_pitch_number',
       'balls_x_bat_score', 'balls_x_fld_score', 'strikes_x_outs_when_up',
       'strikes_x_inning', 'strikes_x_pitch_number', 'strikes_x_bat_score',
       'strikes_x_fld_score', 'outs_when_up_x_inning',
       'outs_when_up_x_pitch_number', 'outs_when_up_x_bat_score',
       'outs_when_up_x_fld_score', 'inning_x_pitch_number',
       'inning_x_bat_score', 'inning_x_fld_score', 'pitch_number_x_bat_score',
       'pitch_number_x_fld_score', 'bat_score_x_fld_score'],
      dtype='object')
     stand  p_throws  balls  strikes  game_year  outs_when_up  inning  \
0        1         0      1        2       2023             2    

In [41]:
prediction_df

,Pitch Type,Probability,Predicted Effective Speed
0,pitch_type_CH,0.26,87.041
1,pitch_type_FF,0.14,95.164
2,pitch_type_SI,0.05,93.684
3,pitch_type_SL,0.55,84.658
